In [63]:
stock = ["euro:10", "materiel:0", "produit:0", "client_content:0"]

process = ["achat_materiel:(euro:8):(materiel:1):10",
           "realisation_produit:(materiel:1):(produit:1):30",
           "livraison:(produit:1):(client_content:1):20" ]

optimize = ["optimize:(time;client_content)"]

In [64]:
class CustomError(Exception):
    pass
        
class ErrorInput(CustomError):
    def __init__(self,*args,**kwargs):
        Exception.__init__(self,*args,**kwargs)

class ErrorOutput(CustomError):
    def __init__(self,*args,**kwargs):
        Exception.__init__(self,*args,**kwargs)

class KRPError(CustomError):
    def __init__(self,*args,**kwargs):
        Exception.__init__(self,*args,**kwargs)

In [85]:
class Config_structure():
    
    def __init__(self):
        self.stock = {}
        self.process = {}
        self.optimize = []

class Process():
    
    def __init__(self):
        self.name = ""
        self.duration = 0
        self.input = {}
        self.output = {}
        self.cycle_ending = 0

class Env():
    
    def __init__(self, conf):
        self.cycle = 0
        self.conf = conf
        self.active_process = []
    
    def add_output(self, process):
        for stock, quant in process.output.items():
            self.conf.stock[stock] += quant
    
    def update_cycle(self, new_cycle):
        self.cycle = new_cycle
        for process in self.active_process:
            if process.cycle_ending <= new_cycle:
                self.add_output(process)
                del process
    
    def process(self, name):
        new_process = Process()
        ref_process = self.conf.process[name]
        for stock, quant in ref_process.input.items():
            self.conf.stock[stock] -= quant
            if self.conf.stock[stock] < 0:
                raise KRPError("Algorithm Error at cycle {} on process '{}': it's impossible to use more stock than available".format(self.cycle, name))
        new_process.output = ref_process.output
        new_process.cycle_ending = self.cycle + ref_process.duration
        self.active_process.append(new_process)

In [88]:
def create_debug_process(conf):
    # Process 1 "achat_materiel:(euro:8):(materiel:1):10"
    p = Process()
    p.name = "achat_materiel"
    p.duration = 10
    p.input = {"euro":8}
    p.output = {"materiel":1}
    conf.process["achat_materiel"] = p
    
    # Process 2 "realisation_produit:(materiel:1):(produit:1):30"
    p = Process()
    p.name = "realisation_produit"
    p.duration = 30
    p.input = {"materiel":1}
    p.output = {"produit":1}
    conf.process["realisation_produit"] = p
    
    # Process 3 "livraison:(produit:1):(client_content:1):20"
    p = Process()
    p.name = "livraison"
    p.duration = 20
    p.input = {"produit":1}
    p.output = {"client_content":1}
    conf.process["livraison"] = p
    
    
    
def read_config(stock, process):
    conf = Config_structure()
    for line in stock:
        single_stock = line.split(':')
        if len(single_stock) != 2:
            raise ErrorInput("Stock Error: Stock bad declaration")
        label = single_stock[0]
        try:
            quant = int(single_stock[1])
        except:
            raise ErrorInput("Stock Error: Quantity should be a valid integer")
        conf.stock[label] = quant
#     for line in process:
#         add_process(conf, line) 
        create_debug_process(conf)
    return conf

def get_action(action):
    tab = action.split(':')
    if len(tab) != 2:
        raise ErrorOutput("KRP Error: Bad output from the KRP_SIM trace")
    try:
        cycle = int(tab[0])
    except:
        raise ErrorOutput("KRP Error: Cycle should be a valid integer")
    return cycle, tab[1]

def verify(conf, actions):
    env = Env(conf)
    for action in actions:
        cycle, name = get_action(action)
        env.update_cycle(cycle)
        env.process(name)
    

In [87]:
stock = ["euro:10", "materiel:0", "produit:0", "client_content:0"]
actions = ["0:achat_materiel", "11:realisation_produit", "41:livraison"]
try:
    t = read_config(stock, process)
    verify(t, actions)
    print("VERIFICATION OK")
except CustomError as err:
    print(err)

VERIFICATION OK


In [89]:
"euro:10".count(':')

1

In [100]:
t = "euro:8"

In [102]:
t.split(':')

['euro', '8']

In [103]:
int('32\n')

32